In [2]:
import numpy as np
from scipy.optimize import minimize, rosen
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV
#from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel, RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, Sum, Product
from sklearn.gaussian_process.kernels import ConstantKernel as C, WhiteKernel as W, RBF, Matern, RationalQuadratic as RQ, ExpSineSquared as ESS, DotProduct as DP, Sum, Product
import warnings
#warnings.filterwarnings("ignore")

#===========================================================================================================#
#Obtaining initial values

np.random.seed(10003)
samples = 12
concentrations = 4
X = np.random.random(samples*concentrations).reshape(samples, concentrations)*10
y = rosen(X.T) * 1e-8

for i in range(len(y)):
    print(f'Sample {i+1}: {X[i].round(3)}, {y[i].round(6)}')


#===========================================================================================================#
#Parameters to Test

def make_bounds(lower, upper):
    bounds = []
    for low in lower:
        for up in upper:
                bounds.append((low, up))
    return bounds

all_kernels = []

kernel = [C(), W(), RBF(), Matern(), RQ(), ESS(), DP()]
operation = [Sum, Product]

upper_bound1 = [1e4, 1e5]
upper_bound2 = [1e4, 1e5]
lower_bound1 = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
lower_bound2 = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]

            
kern1_bounds = make_bounds(lower_bound1, upper_bound1)
kern2_bounds = make_bounds(lower_bound2, upper_bound2)

print(kern1_bounds, kern2_bounds)

#===========================================================================================================#
#Cross fold testing

model = GaussianProcessRegressor(normalize_y=True)

param_grid = [
    {'kernel': [Product(Matern(length_scale=1e+04, nu=1.5), DP(sigma_0=1e+04))],
     'kernel__k1__length_scale_bounds': kern1_bounds,
     'kernel__k2__sigma_0_bounds': kern2_bounds}
]

param_search = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=11, n_jobs=-1)

param_search.fit(X, -y)

best_parameters = param_search.best_params_
best_score = param_search.best_score_

print(best_parameters)
print(best_score)

Sample 1: [9.206 5.806 9.795 7.452], 0.014637
Sample 2: [9.123 5.265 8.88  5.74 ], 0.01178
Sample 3: [0.661 2.222 3.34  7.025], 2.3e-05
Sample 4: [2.022 2.641 2.202 2.139], 3.2e-05
Sample 5: [0.501 2.135 3.23  1.475], 8.6e-05
Sample 6: [0.363 1.749 2.184 1.032], 1.7e-05
Sample 7: [5.306 5.682 0.108 5.365], 0.00157
Sample 8: [0.608 8.135 1.662 2.234], 0.004223
Sample 9: [5.049 3.675 4.728 4.297], 0.000879
Sample 10: [7.569 8.265 5.136 5.122], 0.006847
Sample 11: [0.16  6.248 5.637 4.814], 0.001882
Sample 12: [3.649 6.6   2.837 8.771], 0.001705
[(1e-05, 10000.0), (1e-05, 100000.0), (0.0001, 10000.0), (0.0001, 100000.0), (0.001, 10000.0), (0.001, 100000.0), (0.01, 10000.0), (0.01, 100000.0), (0.1, 10000.0), (0.1, 100000.0), (1.0, 10000.0), (1.0, 100000.0), (10.0, 10000.0), (10.0, 100000.0), (100.0, 10000.0), (100.0, 100000.0), (1000.0, 10000.0), (1000.0, 100000.0)] [(1e-05, 100000.0), (0.0001, 100000.0), (0.001, 100000.0), (0.01, 100000.0), (0.1, 100000.0), (1.0, 100000.0), (10.0, 100000.

d:\Coding\Python\Python311\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__sigma_0 is close to the specified lower bound 1.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


{'kernel': Matern(length_scale=1e+04, nu=1.5) * DotProduct(sigma_0=1e+04), 'kernel__k1__length_scale': 10000.0, 'kernel__k1__nu': 1.5, 'kernel__k2__sigma_0': 10000.0}
-5.981584114117213e-06


{'kernel': 1**2 * RBF(length_scale=1) + DotProduct(sigma_0=1)}
-5.534414196866314e-06

{'kernel': Matern(length_scale=1e+04, nu=1.5) * DotProduct(sigma_0=1e+04), 'kernel__k1__length_scale_bounds': (100.0, 10000.0), 'kernel__k2__sigma_0_bounds': (1.0, 100000.0)}
-5.915878171737609e-06

{'kernel': 31.6**2 * RBF(length_scale=10) + DotProduct(sigma_0=1e+04), 'kernel__k1__constant_value_bounds': (10.0, 10000.0), 'kernel__k2__k1__length_scale_bounds': (10.0, 100.0), 'kernel__k2__k2__sigma_0_bounds': (0.001, 10000.0)}
        'kernel': [Product(C(31.6**2), Sum(RBF(length_scale=10), DP(sigma_0=1e+04)))],
        'kernel__k1__constant_value_bounds': kern1_bounds,
        'kernel__k2__k1__length_scale_bounds': kern2_bounds,
        'kernel__k2__k2__sigma_0_bounds': kern3_bounds
kernel = 31.6**2 * RBF(length_scale=10, constant_value_bounds=(10.0, 10000.0)) + DotProduct(sigma_0=1e+04, sigma_0_bounds=(0.001, 10000.0))
-5.152893145395277e-06